In [1]:
##
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import geopandas
import geopandas as gpd
import matplotlib.pyplot as plt
import pyproj
import math
import os

In [2]:
oldcard1=pd.read_csv(r'F:\扬州\oldcard1_6.csv')
oldcard1

FileNotFoundError: [Errno 2] No such file or directory: 'F:\\扬州\\oldcard1_6.csv'

In [3]:
oldcard1.sort_values(by='时间')

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期
46474,46474,9616900730087686,6,20220606125005,1.5,6220,622,45005,20220606
46475,46475,9616900730122121,6,20220606125011,1.5,6220,622,45011,20220606
46476,46476,9616900721527642,6,20220606130213,1.5,6220,622,50213,20220606
46294,46294,9616900730129190,6,20220606131042,1.5,6270,627,51042,20220606
7501,7501,9616900721529168,101,20220606131048,1.6,880,88,51048,20220606
...,...,...,...,...,...,...,...,...,...
1118,1118,9616900716270379,101,20220607062659,0.0,50,5,982659,20220606
29101,29101,9616900721961989,101,20220607062706,0.0,201,20,982706,20220606
29102,29102,9616900103188114,101,20220607062709,0.0,201,20,982709,20220606
16872,16872,0,101,20220607064146,1.6,11,1,984146,20220606


In [4]:
#线路索引
IDSY = oldcard1.drop_duplicates(subset=['线路编号'],keep='first')
IDSY.sort_values(by='线路编号',inplace=True)
IDSY.reset_index(drop=True,inplace=True) 
IDSY

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606
...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606


In [5]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框

IDSY['段开始时间']=None
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0


t1=40000+0*10000
t2=40000+0*10000+3000
#t2=i*100000+6000
##筛选时间段内的所有刷卡数据
listt=[]   
for g in range(0,len(oldcard1)):
    TIME=int(oldcard1['时间'].iloc[g])
    if TIME>=t1 and TIME<=t2:
        listt.append(g)
oldcard2=oldcard1.loc[listt,:]
oldcard2.reset_index(drop=True,inplace=True) 
print('oldcard2')
print(oldcard2)
##按照线路名统计每条线上的刷卡数据
for i in range(0,len(IDSY)):
    IDSY['段开始时间'].iloc[i]=t1
    tsum=0
    list2=[]
    for k in range(0,len(oldcard2)):
        id1=oldcard2['线路编号'].iloc[k]
        id2=IDSY['线路编号'].iloc[i] 
        
        if id1==id2:                  
            tsum = tsum+1
            IDSY['总客运量'].iloc[i]=tsum
            list2.append(k)
    oldcard3=oldcard2.loc[list2,:]
    #print('oldcard3')
    #print(oldcard3)
    ##线路相同的情况下的刷卡类型统计
    oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
    #print('oldcard4')
    #print(oldcard4)
    osum=len(oldcard4)
    IDSY['老年卡客运量'].iloc[i]=osum
    fosum=tsum-osum
    IDSY['其他卡客运量'].iloc[i]=fosum
ALLIDSY=IDSY       
ALLIDSY

oldcard2
Empty DataFrame
Columns: [Unnamed: 0, 卡号, 卡类型, 交易时间需减去8小时, 交易金额, 线路号, 线路编号, 时间, 日期]
Index: []


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,40000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,40000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,40000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,40000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,40000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,40000,0,0,0
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,40000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,40000,0,0,0
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,40000,0,0,0


In [6]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
for i in range(1,12):
    t1=40000+i*10000
    t2=40000+i*10000+3000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    ##按照线路名统计每条线上的刷卡数据
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t1
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
      ##  print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY=ALLIDSY.append(IDSY)      
ALLIDSY

oldcard2
    Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0         1681  9616900721773904    6  20220606132121   1.0   150    15   
1         1682  9616900721606085    6  20220606132126   1.0   150    15   
2         1683  9616900722020389    6  20220606132130   1.0   150    15   
3         1684  9616900722024117   11  20220606132143   0.0   150    15   
4         1685  9616900721771139    6  20220606132154   1.0   150    15   
..         ...               ...  ...             ...   ...   ...   ...   
58       46478  9616900721786882  101  20220606131416   2.4  6220   622   
59       46479  9616900722105673  101  20220606131419   2.4  6220   622   
60       46480  9616900722161922  101  20220606131810   2.4  6220   622   
61       46481  9616900711006831  101  20220606131928   2.4  6220   622   
62       46539  9616900730130891    6  20220606132138   1.5  6220   622   

       时间        日期  
0   52121  20220606  
1   52126  20220606  
2   52130  20220606  
3 

oldcard2
     Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0            19  9616900721960465   11  20220606202653   0.0  1020   102   
1            38                 0  101  20220606202027   0.0   860    86   
2            39  9616900721517472   11  20220606202319   0.0   860    86   
3            40  9616900722021476  101  20220606202706   0.0   860    86   
4           153  9616900722228927    6  20220606200030   1.0   770    77   
..          ...               ...  ...             ...   ...   ...   ...   
930       50306  9616900730085078    6  20220606200009   1.0  2320   232   
931       50321  9616900722230309    6  20220606200106   1.0  2190   219   
932       50426  9616900722192377   11  20220606201934   0.0   260    26   
933       50427  9616900720190417   11  20220606202232   0.0   260    26   
934       50443  9616900730030460   10  20220606201309   0.0  2800   280   

         时间        日期  
0    122653  20220606  
1    122027  20220606  
2    1

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,50000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,50000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,50000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,50000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,150000,0,0,0
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,150000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,150000,5,0,5
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,150000,3,2,5


In [7]:
ALLIDSY3=ALLIDSY
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=0
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
for i in range(0,8):
    t1=920000+i*10000
    t2=920000+i*10000+3000
    t3=40000+(i+12)*10000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    ##按照线路名统计每条线上的刷卡数据
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t3
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
        #print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY3=ALLIDSY3.append(IDSY)      
ALLIDSY3

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             51  9616900722083426   11  20220607000043   0.0   860    86   
1             58                 0  101  20220607000006   1.6   160    16   
2             59  9616900721938335    6  20220607000537   0.0   160    16   
3             60  9616900721960182   11  20220607000541   0.0   160    16   
4             61  9616900716246306  101  20220607001324   1.6   160    16   
...          ...               ...  ...             ...   ...   ...   ...   
1961       50386  9616900722246540    6  20220607000328   1.0  2190   219   
1962       50457  9616900716122013  101  20220607002247   1.6  2800   280   
1963       50465  9616900722181912    6  20220607001903   0.0   500    50   
1964       50466                 0  101  20220607002718   1.6   500    50   
1965       50493  9616900722263271    2  20220607001436   1.6    30     3   

          时间        日期  
0     920043  20220606  
1     920006  20

oldcard2
Empty DataFrame
Columns: [Unnamed: 0, 卡号, 卡类型, 交易时间需减去8小时, 交易金额, 线路号, 线路编号, 时间, 日期]
Index: []


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,50000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,50000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,50000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,50000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,230000,0,0,0
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,230000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,230000,0,0,0
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,230000,0,0,0


In [8]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=None
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
#t1=40000+0*10000
t1=40000+0*10000+3000
t2=40000+0*10000+6000
#筛选时间段内的所有刷卡数据
listt=[]   
for g in range(0,len(oldcard1)):
    TIME=int(oldcard1['时间'].iloc[g])
    if TIME>=t1 and TIME<=t2:
        listt.append(g)
oldcard2=oldcard1.loc[listt,:]
oldcard2.reset_index(drop=True,inplace=True) 
print('oldcard2')
print(oldcard2)
##按照线路名统计每条线上的刷卡数据
for i in range(0,len(IDSY)):
    IDSY['段开始时间'].iloc[i]=t1
    tsum=0
    list2=[]
    for k in range(0,len(oldcard2)):
        id1=oldcard2['线路编号'].iloc[k]
        id2=IDSY['线路编号'].iloc[i] 
        if id1==id2:                  
            tsum = tsum+1
            IDSY['总客运量'].iloc[i]=tsum
            list2.append(k)
    oldcard3=oldcard2.loc[list2,:]
    #print('oldcard3')
    #print(oldcard3)
    ##线路相同的情况下的刷卡类型统计
    oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
    #print('oldcard4')
    #print(oldcard4)
    osum=len(oldcard4)
    IDSY['老年卡客运量'].iloc[i]=osum
    fosum=tsum-osum
    IDSY['其他卡客运量'].iloc[i]=fosum
ALLIDSY2=IDSY       
ALLIDSY2

oldcard2
   Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号     时间  \
0       46474  9616900730087686    6  20220606125005   1.5  6220   622  45005   
1       46475  9616900730122121    6  20220606125011   1.5  6220   622  45011   

         日期  
0  20220606  
1  20220606  


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,43000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,43000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,43000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,43000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,43000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,43000,0,0,0
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,43000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,43000,0,0,0
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,43000,2,0,2


In [9]:
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
for i in range(1,12):
    t1=40000+i*10000+3000
    t2=40000+i*10000+6000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    ##按照线路名统计每条线上的刷卡数据
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t1
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
      ##  print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY2=ALLIDSY2.append(IDSY)      
ALLIDSY2

oldcard2
     Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0           260  9616900730132775    6  20220606134848   1.0  2120   212   
1          1301  9616900730089842  101  20220606134750   1.6  2320   232   
2          1302  9616900730127492    6  20220606134753   1.0  2320   232   
3          1303  9616900730093636    6  20220606135113   1.0  2320   232   
4          1304  9616900730093358    6  20220606135115   1.0  2320   232   
..          ...               ...  ...             ...   ...   ...   ...   
340       50139  9616900721518456  101  20220606135839   1.6  2210   221   
341       50140  9616900730122843    6  20220606135843   1.0  2210   221   
342       50291  9616900730122724    6  20220606134856   1.0  2320   232   
343       50292  9616900730078102   11  20220606134859   0.0  2320   232   
344       50503  9616900730061276   11  20220606134142   0.0  2210   221   

        时间        日期  
0    54848  20220606  
1    54750  20220606  
2    5475

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             41                 0   11  20220606203329   0.0   860    86   
1            181  9616900720234844   10  20220606204857   0.0   380    38   
2            182  9616900722252455    6  20220606205325   1.0   380    38   
3            200                 0  101  20220606205423   1.6   160    16   
4            201  9616900722199991    6  20220606205651   0.0   160    16   
...          ...               ...  ...             ...   ...   ...   ...   
1024       50307  9616900730079681   11  20220606205839   0.0  2320   232   
1025       50334  9616900730060340   11  20220606205330   0.0  2220   222   
1026       50335  9616900730071163   11  20220606205758   0.0  2220   222   
1027       50336  9616900730064395    4  20220606205840   1.0  2220   222   
1028       50337  9616900730064244    4  20220606205934   1.0  2220   222   

          时间        日期  
0     123329  20220606  
1     124857  20

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,53000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,53000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,53000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,53000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,53000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,153000,1,0,1
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,153000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,153000,2,3,5
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,153000,7,8,15


In [10]:
ALLIDSY4=ALLIDSY2
##以30分钟为粒度统计老年卡和其他卡
##在此基础上新建统计框
IDSY['段开始时间']=0
IDSY['老年卡客运量']=0
IDSY['其他卡客运量']=0
IDSY['总客运量']=0
for i in range(0,8):
    t1=920000+i*10000+3000
    t2=920000+i*10000+6000
    t3=40000+(i+12)*10000+3000
    #t2=i*100000+6000
    ##筛选时间段内的所有刷卡数据
    listt=[]   
    for g in range(0,len(oldcard1)):
        TIME=int(oldcard1['时间'].iloc[g])
        if TIME>=t1 and TIME<=t2:
            listt.append(g)
    oldcard2=oldcard1.loc[listt,:]
    oldcard2.reset_index(drop=True,inplace=True) 
    print('oldcard2')
    print(oldcard2)
    ##按照线路名统计每条线上的刷卡数据
    IDSY['段开始时间']=0
    IDSY['老年卡客运量']=0
    IDSY['其他卡客运量']=0
    IDSY['总客运量']=0
    for i in range(0,len(IDSY)):

        IDSY['段开始时间'].iloc[i]=t3
        tsum=0
        list2=[]
        for k in range(0,len(oldcard2)):
            id1=oldcard2['线路编号'].iloc[k]
            id2=IDSY['线路编号'].iloc[i] 
            if id1==id2:                  
                tsum = tsum+1
                IDSY['总客运量'].iloc[i]=tsum
                list2.append(k)
        oldcard3=oldcard2.loc[list2,:]
        #print('oldcard3')
        #print(oldcard3)
        ##线路相同的情况下的刷卡类型统计
        oldcard4=oldcard3[(oldcard3['卡类型']==6)|(oldcard3['卡类型']==11)]
        #print('oldcard4')
        #print(oldcard4)
        osum=len(oldcard4)
        IDSY['老年卡客运量'].iloc[i]=osum
        fosum=tsum-osum
        IDSY['其他卡客运量'].iloc[i]=fosum
    ALLIDSY4=ALLIDSY4.append(IDSY)      
ALLIDSY4

oldcard2
      Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额   线路号  线路编号  \
0             32  9616900720236756  101  20220607004622   1.6  1020   102   
1             67  9616900721717232    6  20220607003052   1.0   160    16   
2             68                 0   10  20220607003057   0.0   160    16   
3             69  9616900721608781  101  20220607003107   0.0   160    16   
4             70  9616900720244041  101  20220607004725   1.6   160    16   
...          ...               ...  ...             ...   ...   ...   ...   
2211       50412  9616900730123017    6  20220607003120   1.0  2220   222   
2212       50413  9616900730093715    6  20220607003829   1.0  2220   222   
2213       50414  9616900730071577   11  20220607004342   0.0  2220   222   
2214       50415  9616900730089340   11  20220607004348   0.0  2220   222   
2215       50467  9616900721977189    6  20220607003359   1.0   500    50   

          时间        日期  
0     924622  20220606  
1     923052  20

oldcard2
   Unnamed: 0                卡号  卡类型      交易时间需减去8小时  交易金额  线路号  线路编号      时间  \
0       16872                 0  101  20220607064146   1.6   11     1  984146   
1       43448  9616900720319792  101  20220607064411   0.0  601    60  984411   

         日期  
0  20220606  
1  20220606  
oldcard2
Empty DataFrame
Columns: [Unnamed: 0, 卡号, 卡类型, 交易时间需减去8小时, 交易金额, 线路号, 线路编号, 时间, 日期]
Index: []


,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,53000,0,0,0
1,10776,9616900720051413,11,20220606143413,0.0,20,2,63413,20220606,53000,0,0,0
2,22575,9616900722218174,6,20220606150051,0.0,30,3,70051,20220606,53000,0,0,0
3,741,9616900722237394,6,20220606141252,1.0,50,5,61252,20220606,53000,0,0,0
4,32621,9616900721623908,6,20220606144732,0.0,60,6,64732,20220606,53000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,4265,9616900730135218,6,20220606145947,0.5,3870,387,65947,20220606,233000,0,0,0
161,22668,9616900716119692,101,20220606203106,0.8,3880,388,123106,20220606,233000,0,0,0
162,234,9616900730032386,10,20220606154845,0.0,6120,612,74845,20220606,233000,0,0,0
163,1652,0,101,20220606133311,2.4,6220,622,53311,20220606,233000,0,0,0


In [16]:
ALLIDSY3.to_csv(r'F:\扬州\NEWALLIDSY3_6.csv')
ALLIDSY4.to_csv(r'F:\扬州\NEWALLIDSY4_6.csv')

In [17]:
ALL=ALLIDSY3
ALL=ALL.append(ALLIDSY4)
ALL.sort_values(by='段开始时间')

,Unnamed: 0,卡号,卡类型,交易时间需减去8小时,交易金额,线路号,线路编号,时间,日期,段开始时间,老年卡客运量,其他卡客运量,总客运量
0,3214,9616900721789858,11,20220606143236,0.0,10,1,63236,20220606,50000,0,0,0
59,1532,9616900720342478,6,20220606155739,0.0,720,72,75739,20220606,50000,0,0,0
58,736,9616900720360979,6,20220607015845,1.0,710,71,935845,20220606,50000,0,0,0
57,378,0,101,20220606152123,1.6,700,70,72123,20220606,50000,0,0,0
56,7780,9616900721934474,101,20220606171153,1.6,680,68,91153,20220606,50000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,7780,9616900721934474,101,20220606171153,1.6,680,68,91153,20220606,233000,0,0,0
57,378,0,101,20220606152123,1.6,700,70,72123,20220606,233000,0,0,0
58,736,9616900720360979,6,20220607015845,1.0,710,71,935845,20220606,233000,0,0,0
60,1286,9616900722205676,11,20220606152609,0.0,750,75,72609,20220606,233000,0,0,0


In [18]:

ALL=ALL.loc[:,['段开始时间','线路编号','老年卡客运量','其他卡客运量','总客运量']]
ALL['30min开始时间']=0
for i in range(0,len(ALL)):
    T=ALL['段开始时间'].iloc[i]
    t=str(T)
    if len(t)==5:
        TS=t[-2:]
        TM=t[-4:-2]
        TH=t[0]
    if len(t)==6:
        TS=t[-2:]
        TM=t[-4:-2]
        TH=t[:2]
    t=TH+'：'+TM+'：'+TS
    ALL['30min开始时间'].iloc[i]=t

In [19]:
ALL

,段开始时间,线路编号,老年卡客运量,其他卡客运量,总客运量,30min开始时间
0,50000,1,0,0,0,5：00：00
1,50000,2,0,0,0,5：00：00
2,50000,3,0,0,0,5：00：00
3,50000,5,0,0,0,5：00：00
4,50000,6,0,0,0,5：00：00
...,...,...,...,...,...,...
160,233000,387,0,0,0,23：30：00
161,233000,388,0,0,0,23：30：00
162,233000,612,0,0,0,23：30：00
163,233000,622,0,0,0,23：30：00


In [20]:
ALL.to_excel(r'F:\扬州\30min刷卡统计_6(1).xlsx')